In [25]:
%matplotlib widget

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [27]:
"""
Algorithm for generating small world:
1) Randomly position the nodes in a network
2) Connect the nodes to the k closest other nodes
3) Then reshuffle the edge connections like in regular


"""

"""# How to rewire the graph with a certain probability:
for j in range(1, k // 2+1): # outer loop is neighbors
    targets = nodes[j:] + nodes[0:j] # first j nodes are now last in list
    # inner loop in node order
    for u,v in zip(nodes,targets):
        if random.random() < p:
            w = random.choice(nodes)
            # Enforce no self-loops or multiple edges
            while w == u or G.has_edge(u, w):
                w = random.choice(nodes)
                if G.degree(u) >= n-1:
                    break # skip this rewiring
            else:
                G.remove_edge(u,v)
                G.add_edge(u,w)
return G"""

'# How to rewire the graph with a certain probability:\nfor j in range(1, k // 2+1): # outer loop is neighbors\n    targets = nodes[j:] + nodes[0:j] # first j nodes are now last in list\n    # inner loop in node order\n    for u,v in zip(nodes,targets):\n        if random.random() < p:\n            w = random.choice(nodes)\n            # Enforce no self-loops or multiple edges\n            while w == u or G.has_edge(u, w):\n                w = random.choice(nodes)\n                if G.degree(u) >= n-1:\n                    break # skip this rewiring\n            else:\n                G.remove_edge(u,v)\n                G.add_edge(u,w)\nreturn G'

In [28]:
n = 10
k = 4
random_locations = np.random.uniform(0,1,(n,3))
edges_to_add = []
for i,vert in enumerate(random_locations):
    local_distances = np.linalg.norm(random_locations - vert,axis=1)
    smallest_distance_indices = np.argsort(local_distances)[1:k+1]
    edges_to_add += [(i,v) for v in smallest_distance_indices]

In [29]:
import random

def watts_strogatz_graph_smallworld_biological_old(n,k,p):
    G = nx.Graph()
    
    #generate the random loations and edges
    random_locations = np.random.uniform(0,1,(n,3))
    total_edges_to_add = []
    total_edges_dictionary = dict()
    for i,vert in enumerate(random_locations):
        local_distances = np.linalg.norm(random_locations - vert,axis=1)
        #print("local_distances = " + str(local_distances))
        smallest_distance_indices = np.argsort(local_distances)
        #possible_edges = [(i,v) for v in smallest_distance_indices]
        not_already_added = [(i,v) for v in smallest_distance_indices if (v,i) not in total_edges_to_add]
        #print("not_already_added = " + str(not_already_added))
        if len(not_already_added) <= 1:
            print("No new connections available for node {i}")
        if len(not_already_added) <= k:
            edges_to_add = not_already_added[1:]
        else:
            edges_to_add = not_already_added[1:k+1]
        #print("edges_to_add = " + str(edges_to_add))
        total_edges_to_add += edges_to_add
        edges_dictionary = dict([((i,v),local_distances[v]) for i,v in edges_to_add])
        total_edges_dictionary.update(edges_dictionary)
        

    #setting the edge and node attributes
    nodes = list(range(n))
    G.add_nodes_from(nodes)
    G.add_edges_from(total_edges_to_add)
    nx.set_edge_attributes(G,  total_edges_dictionary,"length")
    nx.set_node_attributes(G, dict([(j,random_locations[j,:]) 
                          for j,node in enumerate(G.nodes)]), 'locations')
    
    # How to rewire the graph with a certain probability:
    for j in range(1, k // 2+1): # outer loop is neighbors
        
        targets = nodes[j:] + nodes[0:j] # first j nodes are now last in list
        # inner loop in node order
        #print("nodes = " + str(nodes))
        #print("targets = " + str(targets))
        for u,v in zip(nodes,targets):
            if G.has_edge(u, v) or G.has_edge(v,u):
                if random.random() < p:
                    w = random.choice(nodes)
                    # Enforce no self-loops or multiple edges
                    while w == u or G.has_edge(u, w):
                        w = random.choice(nodes)
                        if G.degree(u) >= n-1:
                            break # skip this rewiring
                    else:
                        #print(f"Node switched form {(u,v)} to {(u,w)}")
                        G.remove_edge(u,v)
                        G.add_edge(u,w)
        
    
    return G
    
    

In [37]:
import random

def watts_strogatz_graph_smallworld_biological(n,k,p):
    G = nx.Graph()
    
    #generate the random loations and edges
    random_locations = np.random.uniform(0,1,(n,3))
    total_edges_to_add = []
    total_edges_dictionary = dict()
    for i,vert in enumerate(random_locations):
        local_distances = np.linalg.norm(random_locations - vert,axis=1)
        #print("local_distances = " + str(local_distances))
        smallest_distance_indices = np.argsort(local_distances)
        #possible_edges = [(i,v) for v in smallest_distance_indices]
        not_already_added = [(i,v) for v in smallest_distance_indices if (v,i) not in total_edges_to_add]
        #print("not_already_added = " + str(not_already_added))
        if len(not_already_added) <= 1:
            print("No new connections available for node {i}")
        if len(not_already_added) <= k:
            edges_to_add = not_already_added[1:]
        else:
            edges_to_add = not_already_added[1:k+1]
        #print("edges_to_add = " + str(edges_to_add))
        total_edges_to_add += edges_to_add
        edges_dictionary = dict([((i,v),local_distances[v]) for i,v in edges_to_add])
        total_edges_dictionary.update(edges_dictionary)
        

    #setting the edge and node attributes
    nodes = list(range(n))
    G.add_nodes_from(nodes)
    G.add_edges_from(total_edges_to_add)
    nx.set_edge_attributes(G,  total_edges_dictionary,"length")
    nx.set_node_attributes(G, dict([(j,random_locations[j,:]) 
                          for j,node in enumerate(G.nodes)]), 'locations')
    
    # How to rewire the graph with a certain probability:
    for u,v in total_edges_to_add:
        if G.has_edge(u, v) or G.has_edge(v,u):
            if random.random() < p:
                w = random.choice(nodes)
                # Enforce no self-loops or multiple edges
                while w == u or G.has_edge(u, w):
                    w = random.choice(nodes)
                    if G.degree(u) >= n-1:
                        break # skip this rewiring
                else:
                    print(f"Node switched form {(u,v)} to {(u,w)}")
                    G.remove_edge(u,v)
                    G.add_edge(u,w)
        
    
    return G
    
    

In [38]:
import graph_experiments_utils as gu
new_graph = watts_strogatz_graph_smallworld_biological(n=8,k=1,p=0)

gu.graph_network_matplotlib(new_graph,plot_type="3D",layout="locations",locations=None,
                 colors="locations",
                 default_color="blue",
                  colors_edge = None,
                  default_color_edge = "green",
                 plot_title="biological_plausible",
                 return_fig = False,
                             node_size=64,
                            node_label_size=20)

Color list index EDGE 0 is blank so setting  color to default of green


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
import graph_experiments_utils as gu
new_graph = watts_strogatz_graph_smallworld_biological(n=8,k=2,p=0.4)

gu.graph_network_matplotlib(new_graph,plot_type="3D",layout="locations",locations=None,
                 colors="locations",
                 default_color="blue",
                  colors_edge = None,
                  default_color_edge = "green",
                 plot_title="biological_plausible",
                 return_fig = False,
                             node_size=64,
                            node_label_size=20)

Node switched form (0, 3) to (0, 2)
Node switched form (1, 7) to (1, 5)
Node switched form (2, 5) to (2, 1)
Node switched form (4, 6) to (4, 2)
Node switched form (5, 0) to (5, 7)
Color list index EDGE 0 is blank so setting  color to default of green


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Wrappers to apply properties to pre-existing functions